In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

## Load data

In [2]:
adata = sc.read_h5ad("Datasets/tpm_unstranded.h5ad")
print(adata)

AnnData object with n_obs × n_vars = 2008 × 60660
    obs: 'sample_type'
    var: 'gene_name', 'gene_type'


/opt/anaconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


## Process data

In [3]:
df = pd.DataFrame(
    data=adata.X,
    index=adata.obs_names,         # sample IDs as row index
    columns=adata.var_names        # gene_ids as column names
)

In [4]:
df = df.T

In [5]:
df

,00000c20-b875-4821-8d4b-c50d7a7f9879,000b6b94-572d-4d06-a8f4-2e43829f83d4,0052ae83-7ae5-470a-a125-5cd94a9fa9e9,00810078-a4fc-4e1d-bbdc-e5e0d3080025,00ad0370-5339-4136-800c-56d4250af0ab,00ad0370-5339-4136-800c-56d4250af0ab,00ad0370-5339-4136-800c-56d4250af0ab,00bc0fe4-481a-4893-b57e-d5113a6bcce2,0163d0f3-46a7-46d2-8ea3-4bf917db2abd,01703923-fdd7-477b-822a-e1eabb0f2404,...,fe1c2811-57fa-4cd8-b5ea-305bb45aabb3,fe20855f-d3ce-4e02-b2dd-3df15eab44bf,fe22447b-79f6-4a9c-9ad8-04fa22b07bd3,fe79c09e-f0ac-4166-b789-33fa1d2fb9d0,fe9b5010-29bb-4e34-97b5-6ce5b512034d,fec22982-c32a-44bd-a1bb-0a377ed028e7,fefb2a51-9eff-464a-bb88-99b886e31698,ff184d74-848e-4a36-b52a-f1304f0996bc,ffc619ed-9dc5-4364-9384-46c9a7fee273,ffec28dd-cc44-4057-bcc3-ccc0cd2331c2
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003.15,49.9587,27.3874,94.7370,22.8215,22.1771,49.9587,27.3874,54.7515,13.6987,12.0211,...,33.4408,25.4086,60.7756,77.9371,71.2798,22.2000,9.9647,25.8883,13.9513,47.3889
ENSG00000000005.6,0.0581,0.0482,0.0000,0.0000,0.0000,0.0581,0.0482,0.0000,0.2801,0.0832,...,0.2853,0.0467,0.6407,0.0000,0.1742,0.4031,0.2507,2.5320,0.8981,0.0000
ENSG00000000419.13,23.5219,113.4229,236.5221,64.8944,11.9062,23.5219,113.4229,52.4916,46.5384,87.3003,...,163.6979,62.8583,95.1990,89.5629,41.1897,110.4794,48.0235,89.7125,61.1011,110.4907
ENSG00000000457.14,7.0283,12.6289,4.1846,6.6049,20.8787,7.0283,12.6289,2.9306,9.0620,11.9545,...,13.4889,6.5035,51.5478,6.6064,10.2741,11.7020,10.6522,18.9755,13.4719,8.6045
ENSG00000000460.17,6.2785,12.1395,7.6534,7.2963,4.8143,6.2785,12.1395,5.9356,2.9088,11.3553,...,7.4761,2.6110,41.8473,9.7875,10.3665,12.9434,5.8462,11.2682,4.0671,5.6382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288669.1,0.0000,0.0000,0.0000,0.0329,0.0000,0.0000,0.0000,0.0000,0.0148,0.0308,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
ENSG00000288670.1,8.8764,10.3222,3.5208,9.4390,23.8585,8.8764,10.3222,6.3307,6.3413,19.9814,...,16.2530,7.7485,41.6076,5.9724,5.6449,6.5441,6.8592,14.3687,9.7689,10.1980
ENSG00000288671.1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [6]:
df = np.log1p(df)

## Get Highly Variable genes across all samples

In [24]:
n_var = 9000

In [25]:
variances = df.var(axis=1)
variances = variances.sort_values(ascending = False)

In [26]:
top_n_var = variances[0:n_var]
top_n_var_genes = list(top_n_var.index)

In [27]:
top_n_var_genes[0:10]

['ENSG00000202538.1',
 'ENSG00000266019.1',
 'ENSG00000252010.1',
 'ENSG00000238741.1',
 'ENSG00000252835.1',
 'ENSG00000212232.1',
 'ENSG00000200795.1',
 'ENSG00000200087.1',
 'ENSG00000207008.1',
 'ENSG00000205420.11']

## Isolate Cancer Samples and evaluate MAD on them

In [28]:
n_mad = 9000

In [29]:
def mad(series):
    return np.median(np.abs(series - np.median(series)))

In [30]:
categories = adata.obs['sample_type'].unique()
for cat in categories:
    print(cat)

Primary Tumor
Solid Tissue Normal
Primary Tumor, Primary Tumor
Solid Tissue Normal, Solid Tissue Normal, Solid Tissue Normal
Solid Tissue Normal, Solid Tissue Normal
Next Generation Cancer Model
Slides
Metastatic
Recurrent Tumor
Expanded Next Generation Cancer Model
Primary Tumor, Primary Tumor, Primary Tumor, Primary Tumor, Primary Tumor
Not Reported
Primary Tumor, Primary Tumor, Primary Tumor


In [31]:
definitely_cancer_labels = [
    "Primary Tumor",
    "Primary Tumor, Primary Tumor",
    "Primary Tumor, Primary Tumor, Primary Tumor",
    "Primary Tumor, Primary Tumor, Primary Tumor, Primary Tumor, Primary Tumor",
    "Metastatic",
    "Recurrent Tumor",
    "Next Generation Cancer Model",
    "Expanded Next Generation Cancer Model"
]

# Filter the AnnData object to keep only cancer observations
cancer_adata = adata[adata.obs["sample_type"].isin(definitely_cancer_labels)].copy()

# Optional: check the result
print(cancer_adata.shape)
print(cancer_adata.obs["sample_type"].value_counts())

(1584, 60660)
sample_type
Primary Tumor                                                                1490
Primary Tumor, Primary Tumor                                                   64
Metastatic                                                                     16
Next Generation Cancer Model                                                    6
Primary Tumor, Primary Tumor, Primary Tumor                                     4
Recurrent Tumor                                                                 2
Expanded Next Generation Cancer Model                                           1
Primary Tumor, Primary Tumor, Primary Tumor, Primary Tumor, Primary Tumor       1
Name: count, dtype: int64


/opt/anaconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [32]:
cancer_df = pd.DataFrame(
    data=cancer_adata.X,
    index=cancer_adata.obs_names,         # sample IDs as row index
    columns=cancer_adata.var_names        # gene_ids as column names
)

In [33]:
cancer_df = cancer_df.T

In [34]:
cancer_df

,00000c20-b875-4821-8d4b-c50d7a7f9879,000b6b94-572d-4d06-a8f4-2e43829f83d4,0052ae83-7ae5-470a-a125-5cd94a9fa9e9,00810078-a4fc-4e1d-bbdc-e5e0d3080025,00ad0370-5339-4136-800c-56d4250af0ab,00ad0370-5339-4136-800c-56d4250af0ab,00ad0370-5339-4136-800c-56d4250af0ab,00bc0fe4-481a-4893-b57e-d5113a6bcce2,01703923-fdd7-477b-822a-e1eabb0f2404,01977422-7f46-41d2-bc19-ed0093af8f25,...,fdfac8d6-035f-451b-b96d-2cc010c7e015,fe1c2811-57fa-4cd8-b5ea-305bb45aabb3,fe20855f-d3ce-4e02-b2dd-3df15eab44bf,fe22447b-79f6-4a9c-9ad8-04fa22b07bd3,fe79c09e-f0ac-4166-b789-33fa1d2fb9d0,fe9b5010-29bb-4e34-97b5-6ce5b512034d,fec22982-c32a-44bd-a1bb-0a377ed028e7,fefb2a51-9eff-464a-bb88-99b886e31698,ff184d74-848e-4a36-b52a-f1304f0996bc,ffec28dd-cc44-4057-bcc3-ccc0cd2331c2
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003.15,49.9587,27.3874,94.7370,22.8215,22.1771,49.9587,27.3874,54.7515,12.0211,73.2201,...,18.2435,33.4408,25.4086,60.7756,77.9371,71.2798,22.2000,9.9647,25.8883,47.3889
ENSG00000000005.6,0.0581,0.0482,0.0000,0.0000,0.0000,0.0581,0.0482,0.0000,0.0832,0.2047,...,0.5727,0.2853,0.0467,0.6407,0.0000,0.1742,0.4031,0.2507,2.5320,0.0000
ENSG00000000419.13,23.5219,113.4229,236.5221,64.8944,11.9062,23.5219,113.4229,52.4916,87.3003,219.9834,...,55.1145,163.6979,62.8583,95.1990,89.5629,41.1897,110.4794,48.0235,89.7125,110.4907
ENSG00000000457.14,7.0283,12.6289,4.1846,6.6049,20.8787,7.0283,12.6289,2.9306,11.9545,11.7142,...,11.6666,13.4889,6.5035,51.5478,6.6064,10.2741,11.7020,10.6522,18.9755,8.6045
ENSG00000000460.17,6.2785,12.1395,7.6534,7.2963,4.8143,6.2785,12.1395,5.9356,11.3553,9.6585,...,12.1340,7.4761,2.6110,41.8473,9.7875,10.3665,12.9434,5.8462,11.2682,5.6382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288669.1,0.0000,0.0000,0.0000,0.0329,0.0000,0.0000,0.0000,0.0000,0.0308,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
ENSG00000288670.1,8.8764,10.3222,3.5208,9.4390,23.8585,8.8764,10.3222,6.3307,19.9814,7.8270,...,7.3909,16.2530,7.7485,41.6076,5.9724,5.6449,6.5441,6.8592,14.3687,10.1980
ENSG00000288671.1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [35]:
mad_values = cancer_df.apply(mad, axis=1)
top_mad_genes = mad_values.nlargest(n_mad).index

In [36]:
top_mad_genes

Index(['ENSG00000211592.8', 'ENSG00000198886.2', 'ENSG00000198712.1',
       'ENSG00000198804.2', 'ENSG00000198938.2', 'ENSG00000198899.2',
       'ENSG00000087086.15', 'ENSG00000198763.3', 'ENSG00000198727.2',
       'ENSG00000210082.2',
       ...
       'ENSG00000163536.13', 'ENSG00000244586.1', 'ENSG00000138381.10',
       'ENSG00000147799.11', 'ENSG00000157800.18', 'ENSG00000274605.2',
       'ENSG00000163701.19', 'ENSG00000078124.12', 'ENSG00000211448.13',
       'ENSG00000223803.1'],
      dtype='object', name='gene_id', length=9000)

## Get total gene set 

In [37]:
total_gene_set = set(top_n_var_genes).union(set(top_mad_genes))
total_gene_set = list(total_gene_set)
print(f"Total unique genes in union: {len(total_gene_set)}")

Total unique genes in union: 15806


In [38]:
total_gene_file = "total_gene_list_correlated.txt"
with open(total_gene_file, "w") as f:
    for gene in total_gene_set:
         f.write(gene + "\n")
print(f"Selected (possibly correlated) genes saved to {total_gene_file}")

Selected (possibly correlated) genes saved to 'total_gene_list_correlated.txt'


## Remove highly correlated genes

In [39]:
corr_threshold = 0.8

In [40]:
filtered_data = df.loc[total_gene_set]
print(f"Filtered data shape (after union): {filtered_data.shape}")

Filtered data shape (after union): (15806, 2008)


In [41]:
corr_matrix = filtered_data.T.corr().abs()

In [42]:
# Keep those genes that explain the maximum variance
variance_subset = variances.loc[total_gene_set]
sorted_genes = variance_subset.sort_values(ascending=False).index.tolist()

In [43]:
selected_genes = []
removed_genes = set()

for gene in sorted_genes:
    # Won't consider the same gene twice
    if gene in removed_genes:
        continue
    selected_genes.append(gene)
    # Identify genes that are highly correlated with the current gene.
    correlated_genes = corr_matrix.loc[gene]
    # Remove all genes that are above the correlation threshold
    to_remove = correlated_genes[correlated_genes > corr_threshold].index.tolist()
    removed_genes.update(to_remove)

In [44]:
print(f"Genes remaining after correlation filtering: {len(selected_genes)}")
final_gene_file = "final_gene_list_non_correlated.txt"
# Final filtered data with non-redundant genes
final_filtered_data = filtered_data.loc[selected_genes]
print(f"Final filtered data shape: {final_filtered_data.shape}")

with open(final_gene_file, "w") as f:
    for gene in selected_genes:
         f.write(gene + "\n")

print(f"Final selected genes saved to {final_gene_file}")

Genes remaining after correlation filtering: 9363
Final filtered data shape: (9363, 2008)
Final selected genes saved to 'final_gene_list_non_correlated.txt'
